In [3]:
import numpy as np
import pandas as pd

from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error , mean_absolute_error
from sklearn.model_selection import cross_val_score, cross_val_predict , KFold,GridSearchCV

from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, OneHotEncoder

import joblib
import matplotlib.pyplot as plt

In [4]:
df_casas = pd.read_csv('limpio.csv')



In [ ]:
x,y = df_casas.drop(['precio','ventaoalquiler','ciudad_casa','ubicacion','unidadterreno','descripespecif'], axis=1), df_casas['precio']

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.15, random_state=42)



In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder

categorias = ['zona','descripespecif']
numerical_features = ['habitaciones','banos','area','parqueaderos','estrato','tiempodeconstruido','latitud','longitud']

preprocessor = make_column_transformer(
    (OneHotEncoder(drop='if_binary'), categorias),
    remainder='passthrough',
    verbose_feature_names_out=False)

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge
from sklearn.compose import TransformedTargetRegressor
import scipy as sp

model = make_pipeline(
    preprocessor,
    TransformedTargetRegressor(
        regressor=Ridge(alpha=0.1),
        func=np.log10,
        inverse_func=sp.special.exp10
    )
)

In [ ]:
model.fit(x_train, y_train)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

y_pred = model.predict(x_train)

mae = mean_absolute_error(y_train, y_pred)
string_score = f'MAE en train: {mae:.2f}'  
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_test, y_pred)
string_score += f'\nMAE en test: {mae:.2f}'



In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
plt.scatter(y_test, y_pred)
ax.plot([0, 1], [0, 1], transform=ax.transAxes, ls="--", c="red")
plt.text(500000000, 1000000000, string_score)
plt.title('Ridge')
plt.ylabel('Predicciones')
plt.xlabel('Valores reales')